In [1]:
import pandas as pd
df=pd.read_csv("blogs.csv")
df

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism
...,...,...
1995,Xref: cantaloupe.srv.cs.cmu.edu talk.abortion:...,talk.religion.misc
1996,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc
1997,Xref: cantaloupe.srv.cs.cmu.edu talk.origins:4...,talk.religion.misc
1998,Xref: cantaloupe.srv.cs.cmu.edu talk.religion....,talk.religion.misc


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Data    2000 non-null   object
 1   Labels  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [3]:
df['Labels'].value_counts()

,count
Labels,
alt.atheism,100
comp.graphics,100
comp.os.ms-windows.misc,100
comp.sys.ibm.pc.hardware,100
comp.sys.mac.hardware,100
comp.windows.x,100
misc.forsale,100
rec.autos,100
rec.motorcycles,100


In [29]:
 #nlp setup: import required libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [30]:
#text preprocessing: converting to lowercase, tokenizing, removing stopwords, joining again as string
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    text = ' '.join(tokens)
    return text

In [31]:
df['Data'] = df['Data'].apply(preprocess_text)

In [32]:
# Feature extraction using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [33]:

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Data'])
y = df['Labels']

In [37]:
#splitting data,
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
#model training
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [39]:
#make predictions
y_pred = nb_model.predict(X_test)

In [40]:
#Evaluate the model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.47      0.78      0.58        18
           comp.graphics       0.57      0.67      0.62        18
 comp.os.ms-windows.misc       0.78      0.82      0.80        22
comp.sys.ibm.pc.hardware       0.75      0.60      0.67        25
   comp.sys.mac.hardware       0.71      0.57      0.63        21
          comp.windows.x       0.90      0.36      0.51        25
            misc.forsale       0.85      0.61      0.71        18
               rec.autos       0.77      0.94      0.85        18
         rec.motorcycles       0.87      0.81      0.84        16
      rec.sport.baseball       0.75      0.83      0.79        18
        rec.sport.hockey       0.83      1.00      0.91        15
               sci.crypt       0.67      0.95      0.78        19
         sci.electronics       0.44      0.69      0.54        16
                 sci.med       0.88      0.88      0.88        17
         

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7275
Precision: 0.753632814016846
Recall: 0.7275
F1-score: 0.7172599151729275


In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [24]:
def sentiment_analysis(text):
    sentiment = sia.polarity_scores(text)
    if sentiment['compound'] > 0.05:
        return 'Positive'
    elif sentiment['compound'] < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [25]:
df['Sentiment'] = df['Data'].apply(sentiment_analysis)

In [28]:
print(df['Sentiment'].value_counts())
print(df.groupby('Labels')['Sentiment'].value_counts())

Sentiment
Positive    1368
Negative     579
Neutral       53
Name: count, dtype: int64
Labels                    Sentiment
alt.atheism               Positive     59
                          Negative     40
                          Neutral       1
comp.graphics             Positive     86
                          Negative     12
                          Neutral       2
comp.os.ms-windows.misc   Positive     76
                          Negative     21
                          Neutral       3
comp.sys.ibm.pc.hardware  Positive     76
                          Negative     21
                          Neutral       3
comp.sys.mac.hardware     Positive     81
                          Negative     16
                          Neutral       3
comp.windows.x            Positive     78
                          Negative     20
                          Neutral       2
misc.forsale              Positive     83
                          Neutral      10
                          Negative   

In [27]:
df

,Data,Labels,Sentiment
0,path agate uknet mcsun frank frank frank newsg...,alt.atheism,Negative
1,newsgroups path constellation darkside bil bil...,alt.atheism,Positive
2,path uunet olivea decwrl jimh jimh james hogan...,alt.atheism,Negative
3,path usc venice jbrown jbrown newsgroups subje...,alt.atheism,Negative
4,xref newsgroups path rphroy caen uunet pipex u...,alt.atheism,Positive
...,...,...,...
1995,xref path agate uknet mcsun frank frank frank ...,talk.religion.misc,Positive
1996,xref newsgroups path rochester udel purdue mec...,talk.religion.misc,Positive
1997,xref path gatech purdue scharle scharle scharl...,talk.religion.misc,Positive
1998,xref path udel psinntp foster foster newsgroup...,talk.religion.misc,Positive


The Naive Bayes model achieves an:

- Accuracy: 0.7275 (72.75%)
- Precision: 0.7536 (75.36%)
- Recall: 0.7275 (72.75%)
- F1-score: 0.7173 (71.73%)

The model performs reasonably well on the text classification task. The precision being higher than recall suggests that the model is more precise in its predictions but might miss some relevant instances.

Discussion
- The Naive Bayes model is effective for text classification on the blog posts dataset.
- The model's performance metrics indicate a balanced trade-off between precision and recall.
- Challenges in classification might include handling complex linguistic structures or category imbalances.

Sentiment Analysis Implications
- Sentiment analysis provides insights into the emotional tone of the blog posts.
- Understanding sentiment distribution across categories can help in content analysis and targeting specific audiences.

Overall, the Naive Bayes model shows decent performance for categorizing blog posts, and sentiment analysis adds an additional layer of understanding the content.